In [1]:
import os
import json
from collections import Counter, defaultdict
import pandas as pd # 可用於數據檢視 (可選)
from sklearn.model_selection import train_test_split

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig, # 用於量化載入基礎模型
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

print("函式庫導入完成。")

c:\Users\ISLAB-121\.conda\envs\aicup\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


函式庫導入完成。


In [2]:
# Cell 2: 設定參數

# --- 模型相關 ---
# **** 更新基礎模型路徑 ****
BASE_MODEL_PATH = "./models/Qwen3-8B" # <--- 將此路徑更新為您的非 FP8 版本 Qwen3-8B 模型路徑
DEVICE_FOR_MODEL = "cuda" if torch.cuda.is_available() else "cpu"
print(f"將使用的運算設備 (推斷時): {DEVICE_FOR_MODEL}")

# 輸出目錄可以保持不變，或者您也可以根據新的基礎模型調整它們的命名
OUTPUT_DIR_MODEL1 = "./qwen3_8b_ner_original_data_non_fp8" 
OUTPUT_DIR_MODEL2 = "./qwen3_8b_ner_augmented_data_non_fp8"

# --- 資料路徑 (保持不變) ---
ORIGINAL_TEXT_FILE = "dataset/TRAINGING_DATASET_1/Training_Dataset_01/task1_answer.txt"
ORIGINAL_ANNOTATION_FILE = "dataset/TRAINGING_DATASET_1/Training_Dataset_01/task2_answer.txt"
AUGMENTED_TEXT_FILE = "gen_transcriptions_qwen_2step_no_think_target60.txt" # 假設這是您滿意的增強資料
AUGMENTED_ANNOTATION_FILE = "gen_annotations_qwen_2step_no_think_target60.txt"

# --- 訓練參數 (保持不變，但您可以根據需要調整) ---
NUM_TRAIN_EPOCHS_MODEL1 = 3
PER_DEVICE_TRAIN_BATCH_SIZE_MODEL1 = 2 
LEARNING_RATE_MODEL1 = 2e-4

NUM_TRAIN_EPOCHS_MODEL2 = 3 
PER_DEVICE_TRAIN_BATCH_SIZE_MODEL2 = 2
LEARNING_RATE_MODEL2 = 2e-4

# --- LoRA 設定 (保持不變) ---
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05
LORA_TARGET_MODULES = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]

# --- 其他 (保持不變) ---
NULL_ENTITY_REPRESENTATION = "Null"
ENTITY_SEPARATOR = "##"
TYPE_TEXT_SEPARATOR = "\t"

def get_all_entity_types(annotation_file_paths):
    all_types = set()
    for file_path in annotation_file_paths:
        if not os.path.exists(file_path):
            print(f"警告: 獲取實體類型時，檔案 {file_path} 不存在。")
            continue
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    parts = line.strip().split('\t')
                    if len(parts) >= 2: 
                        all_types.add(parts[1].strip())
        except Exception as e:
            print(f"讀取 {file_path} 以獲取實體類型時出錯: {e}")
    sorted_types = sorted(list(all_types))
    print(f"從標註檔案中找到的實體類型: {sorted_types}")
    return sorted_types

ALL_ENTITY_TYPES = get_all_entity_types([ORIGINAL_ANNOTATION_FILE, AUGMENTED_ANNOTATION_FILE])
ALL_ENTITY_TYPES_STR = ", ".join(ALL_ENTITY_TYPES) if ALL_ENTITY_TYPES else "未指定類型 (請檢查標註檔)"

print(f"\n基礎模型路徑更新為: {BASE_MODEL_PATH}")
print(f"所有可能的實體類型: {ALL_ENTITY_TYPES_STR}")

將使用的運算設備 (推斷時): cuda
從標註檔案中找到的實體類型: ['AGE', 'CITY', 'COUNTRY', 'COUNTY', 'DATE', 'DEPARTMENT', 'DISTRICT', 'DOCTOR', 'DURATION', 'FAMILYNAME', 'HOSPITAL', 'ID_NUMBER', 'LOCATION-OTHER', 'MEDICAL_RECORD_NUMBER', 'ORGANIZATION', 'PATIENT', 'PERSONALNAME', 'PHONE', 'PROFESSION', 'PROFSSION', 'ROOM', 'SET', 'STATE', 'STREET', 'TIME', 'URL', 'ZIP']

基礎模型路徑更新為: ./models/Qwen3-8B
所有可能的實體類型: AGE, CITY, COUNTRY, COUNTY, DATE, DEPARTMENT, DISTRICT, DOCTOR, DURATION, FAMILYNAME, HOSPITAL, ID_NUMBER, LOCATION-OTHER, MEDICAL_RECORD_NUMBER, ORGANIZATION, PATIENT, PERSONALNAME, PHONE, PROFESSION, PROFSSION, ROOM, SET, STATE, STREET, TIME, URL, ZIP


In [3]:
# Cell 3: 資料載入與預處理函式

def load_and_prepare_ner_data(text_file_path, annotation_file_path, is_augmented_data_annotation=False):
    """
    載入轉錄稿和標註，並將它們組合成 NER Seq2Seq 任務所需的格式。
    is_augmented_data_annotation: 標記註釋文件是否為增強數據的格式 (Type\tNone\tNone\tText)
    """
    texts = {}
    if not os.path.exists(text_file_path):
        print(f"錯誤: 文本檔案 '{text_file_path}' 不存在。")
        return []
    try:
        with open(text_file_path, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t', 1)
                if len(parts) == 2:
                    filename = os.path.splitext(parts[0].strip())[0]
                    texts[filename] = parts[1].strip()
    except Exception as e:
        print(f"讀取文本檔案 '{text_file_path}' 時出錯: {e}")
        return []

    annotations_by_filename = defaultdict(list)
    if not os.path.exists(annotation_file_path):
        print(f"錯誤: 標註檔案 '{annotation_file_path}' 不存在。")
        # 即使標註檔不存在，也可能要處理純文本（如果允許無標註文本參與）
        # 但對於 NER 任務，通常標註是必要的。這裡我們先假設標註檔必須存在。
        # 如果允許無標註的情況，則需要調整後續邏輯。
        # 目前，如果標註檔不存在，將導致 annotations_by_filename 為空。
    else:
        try:
            with open(annotation_file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    parts = line.strip().split('\t')
                    # 原始標註: filename\tType\tstart\tend\tText
                    # 增強標註: gen_filename\tType\tNone\tNone\tText
                    expected_parts = 5 # 對於增強數據，中間兩個是 "None"
                    if len(parts) == expected_parts:
                        filename = os.path.splitext(parts[0].strip())[0]
                        entity_type = parts[1].strip()
                        # start = parts[2].strip() # 對於Seq2Seq NER，我們不直接使用 start/end
                        # end = parts[3].strip()
                        entity_text = parts[4].strip()
                        
                        if entity_type and entity_text:
                            annotations_by_filename[filename].append(f"{entity_type}{TYPE_TEXT_SEPARATOR}{entity_text}")
        except Exception as e:
            print(f"讀取標註檔案 '{annotation_file_path}' 時出錯: {e}")

    processed_data = []
    for filename, content in texts.items():
        target_entities = annotations_by_filename.get(filename, [])
        if target_entities:
            target_text = ENTITY_SEPARATOR.join(target_entities)
        else:
            target_text = NULL_ENTITY_REPRESENTATION # "Null"
        
        processed_data.append({
            "id": filename,
            "input_text": content, # 這是原始句子
            "target_text": target_text
        })
        
    print(f"從 {os.path.basename(text_file_path)} 和 {os.path.basename(annotation_file_path)} 處理了 {len(processed_data)} 筆資料。")
    return processed_data

print("資料載入與預處理函式定義完成。")

資料載入與預處理函式定義完成。


In [4]:
# Cell 4: 準備資料集 (Model 1 & Model 2)

# 載入原始資料
original_data_list = load_and_prepare_ner_data(ORIGINAL_TEXT_FILE, ORIGINAL_ANNOTATION_FILE)

# 載入增強資料
augmented_data_list = load_and_prepare_ner_data(AUGMENTED_TEXT_FILE, AUGMENTED_ANNOTATION_FILE, is_augmented_data_annotation=True)

# Model 1 資料集: 僅原始資料
model1_data = original_data_list
print(f"\n模型1將使用 {len(model1_data)} 筆原始資料進行訓練。")

# Model 2 資料集: 原始資料 + 增強資料
model2_data = original_data_list + augmented_data_list
print(f"模型2將使用 {len(model2_data)} 筆總資料 (原始 + 增強) 進行訓練。")

# 轉換為 Hugging Face Dataset 物件並進行切分
def create_dataset_dict(data_list, test_size=0.1):
    if not data_list:
        return None
    # 先轉換為 pandas DataFrame 再轉為 Dataset，或者直接從 list of dicts 創建
    dataset = Dataset.from_list(data_list)
    if test_size > 0 and len(dataset) > 1 : # 確保數據量足夠切分
        train_test_split_data = dataset.train_test_split(test_size=test_size, shuffle=True, seed=42)
        return DatasetDict({
            'train': train_test_split_data['train'],
            'validation': train_test_split_data['test'] 
        })
    else: # 如果數據太少或不切分，全部作為訓練集
        return DatasetDict({'train': dataset})


dataset_model1 = create_dataset_dict(model1_data)
dataset_model2 = create_dataset_dict(model2_data)

if dataset_model1:
    print("\n模型1資料集結構:")
    print(dataset_model1)
if dataset_model2:
    print("\n模型2資料集結構:")
    print(dataset_model2)

從 task1_answer.txt 和 task2_answer.txt 處理了 2314 筆資料。
從 gen_transcriptions_qwen_2step_no_think_target60.txt 和 gen_annotations_qwen_2step_no_think_target60.txt 處理了 403 筆資料。

模型1將使用 2314 筆原始資料進行訓練。
模型2將使用 2717 筆總資料 (原始 + 增強) 進行訓練。

模型1資料集結構:
DatasetDict({
    train: Dataset({
        features: ['id', 'input_text', 'target_text'],
        num_rows: 2082
    })
    validation: Dataset({
        features: ['id', 'input_text', 'target_text'],
        num_rows: 232
    })
})

模型2資料集結構:
DatasetDict({
    train: Dataset({
        features: ['id', 'input_text', 'target_text'],
        num_rows: 2445
    })
    validation: Dataset({
        features: ['id', 'input_text', 'target_text'],
        num_rows: 272
    })
})


In [5]:
# Cell 5: Tokenizer 和 Tokenization 函數 (更新版)

#載入 Tokenizer (假設 tokenizer 已在 Cell 2 或此 Cell 前面部分正確載入)
try:
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token 
    print(f"Tokenizer '{BASE_MODEL_PATH}' 載入成功。Pad token ID: {tokenizer.pad_token_id}")
except Exception as e:
    print(f"載入 Tokenizer '{BASE_MODEL_PATH}' 失敗: {e}")
    tokenizer = None

# Tokenization 函數 (針對 Causal LM 微調進行修改)
def tokenize_function_causal_lm(examples):
    if tokenizer is None:
        raise ValueError("Tokenizer 未成功載入。")

    processed_examples = {"input_ids": [], "attention_mask": [], "labels": []}
    
    # 為輸入和輸出序列設定一個統一的最大長度
    # 這個長度需要足夠容納 prompt + target。Qwen 模型通常支持較長上下文。
    # 例如: 1024 (原 prompt) + 256 (原 target) + 一些額外空間 = 1280 或 1536
    # 實際值需要根據您的資料長度分佈和模型能力調整。
    # 為了與之前的設置有所比較，我們暫設為 1280
    # 您可以根據 Qwen3-8B 的能力和您的數據調整，例如 2048, 4096
    COMBINED_MAX_LENGTH = 1280 

    for i in range(len(examples["input_text"])):
        text_content = examples["input_text"][i]
        target_content_str = examples["target_text"][i] # e.g., "Type1\tText1##Type2\tText2" or "Null"

        # 1. 構建輸入模型的完整提示 (不包含答案)
        system_prompt = (
            f"You are an expert Named Entity Recognition (NER) system. "
            f"Your task is to extract all specified entities from the given medical text. "
            f"Output the entities in the format: Type1{TYPE_TEXT_SEPARATOR}Text1{ENTITY_SEPARATOR}Type2{TYPE_TEXT_SEPARATOR}Text2... "
            f"If no entities are found, output exactly '{NULL_ENTITY_REPRESENTATION}'.\n"
            f"The entity types to look for are: {ALL_ENTITY_TYPES_STR}."
        )
        messages_prompt_part = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text_content}
        ]
        
        # apply_chat_template 會加入 assistant turn 的開頭標記, e.g., "<|im_start|>assistant\n"
        # 這部分是 prompt，模型不應該學習預測它
        prompt_string = tokenizer.apply_chat_template(
            messages_prompt_part, 
            tokenize=False, 
            add_generation_prompt=True # 確保 assistant turn 的開頭被加入
        )

        # 2. 構建完整的輸入序列 = prompt + target_answer + eos
        full_example_text = prompt_string + target_content_str + tokenizer.eos_token

        # 3. Tokenize 完整序列
        tokenized_full = tokenizer(
            full_example_text,
            max_length=COMBINED_MAX_LENGTH,
            padding="max_length", # Pad 到 COMBINED_MAX_LENGTH
            truncation=True,
            return_tensors=None, # .map 會處理
        )

        # 4. Tokenize 僅 prompt 部分，以計算其長度用於遮罩 labels
        # 確保這裡的 tokenize 方式與 full_example_text 的前綴部分一致
        # 通常 apply_chat_template 輸出的 string 直接 tokenize 即可
        # 我們需要的是 prompt_string 在 tokenized_full["input_ids"] 中的實際長度
        
        # 為了得到精確的 prompt token 長度，我們 tokenize prompt_string
        # 然後再 tokenize target_content_str + tokenizer.eos_token
        # 比較兩者與 tokenized_full 的關係是更穩健的做法，但較複雜
        # 一個簡化的假設：直接 tokenize prompt_string 得到的長度可用於遮罩
        # 注意：如果 tokenizer 給 prompt_string 和 full_example_text 加不同的開頭 token (e.g. BOS), 會有偏差
        # Qwen 的 apply_chat_template 通常會處理好 BOS/EOS，這裡我們假設 prompt_string 的 token 化結果是 full_example_text 的前綴
        
        tokenized_prompt_only = tokenizer(prompt_string, add_special_tokens=False) # 通常 add_special_tokens=False for parts
                                                                                # 但 apply_chat_template 可能已包含
                                                                                # 為了保險，我們token化完整的prompt_string
        prompt_tokens_length = len(tokenized_prompt_only['input_ids'])
        
        # 驗證一下，如果 prompt_string 本身被 tokenizer 加了 BOS，而 full_example_text 的開頭也是 BOS，
        # 那麼直接用 tokenized_prompt_only 的長度可能不準確，它可能比實際在 full_example_text 中的 prompt 部分多一個 BOS。
        # 更安全的方式：
        temp_tokenized_prompt = tokenizer(prompt_string, max_length=COMBINED_MAX_LENGTH, truncation=True, add_special_tokens=tokenizer.bos_token_id is not None)
        prompt_tokens_length = 0
        # 找到 assistant 回應開始的地方
        # 我們知道 prompt_string 以 assistant 的起始提示結束
        # 而 target_content_str 是 assistant 的實際回應
        # 所以 labels 中，prompt_string 對應的部分都應該是 -100
        
        # 重新考慮 prompt_tokens_length 的計算：
        # 我們要遮罩的是 prompt_string 對應的 token。
        # input_ids 是 tokenized_full['input_ids']
        # labels 初始化為 input_ids 的拷貝
        labels = list(tokenized_full['input_ids'])
        
        # 關鍵：找到 target_content_str 在 tokenized_full['input_ids'] 中開始的位置
        # 這通常緊隨 prompt_string 的 token 之後。
        # 我們假設 prompt_string 的 token 化長度是固定的。
        # tokenized_prompt_only = tokenizer(prompt_string, add_special_tokens=True) # 假設模板會加開頭
        # prompt_tokens_length = len(tokenized_prompt_only['input_ids'])
        # 這一步的 prompt_tokens_length 非常關鍵，要確保它準確地代表了
        # tokenized_full['input_ids'] 中屬於 prompt (不含 target) 的部分。
        # 如果 tokenizer.apply_chat_template 已經包含了所有必要的特殊 token (如 BOS)，
        # 則直接 tokenize(prompt_string) 得到的長度應該是準確的。
        
        # 讓我們用一個更穩健的方式來計算 prompt 的實際 token 數
        # 在 full_example_text = prompt_string + target_content_str + tokenizer.eos_token
        # 我們需要遮蔽 prompt_string 的部分
        
        # Tokenize prompt_string and target_content_str+eos separately without padding/truncation first to see their true lengths
        prompt_part_tokens = tokenizer(prompt_string, add_special_tokens=False).input_ids # Qwen template handles BOS/EOS for turns.
        target_part_tokens = tokenizer(target_content_str + tokenizer.eos_token, add_special_tokens=False).input_ids

        # The actual prompt length within the combined tokenized sequence
        # This assumes that tokenizing (A+B) is roughly tokenizing(A) + tokenizing(B) in terms of where B starts
        # This can be tricky due to merging tokens.
        # A common robust way: tokenize prompt, tokenize response, then check if they match start/end of combined.
        # For now, let's use a direct tokenization of prompt_string for length:
        # The prompt_string *already* contains the prefix for the assistant's turn.
        # So, everything in prompt_string should be masked.
        tokenized_prompt_for_mask_len = tokenizer(prompt_string, add_special_tokens=True) # Use add_special_tokens consistent with how full_example_text is tokenized
        prompt_mask_len = len(tokenized_prompt_for_mask_len['input_ids'])
        
        # 如果 tokenizer 自動在開頭加 BOS，而 prompt_string 的 apply_chat_template 沒顯式加，長度會有差異
        # 最安全的方式是找到 target 在 full_example_text 中的 token 起始位置
        # 但這通常很難完美。我們假設 tokenize(prompt_string) 給出的是需要 mask 的長度。

        # 創建 labels
        current_labels = list(tokenized_full['input_ids']) # 拷貝
        for j in range(len(current_labels)):
            if j < prompt_mask_len : # 遮罩 prompt 部分
                current_labels[j] = -100
            elif tokenized_full['input_ids'][j] == tokenizer.pad_token_id: # 遮罩 padding 部分
                current_labels[j] = -100
        
        processed_examples["input_ids"].append(tokenized_full["input_ids"])
        processed_examples["attention_mask"].append(tokenized_full["attention_mask"])
        processed_examples["labels"].append(current_labels)
        
    return processed_examples

# 對資料集進行 Tokenization
if tokenizer and dataset_model1:
    # 在 map 之前，確保 Cell 2 中的 ALL_ENTITY_TYPES_STR 等已定義
    if 'ALL_ENTITY_TYPES_STR' not in globals() or not ALL_ENTITY_TYPES_STR:
        print("錯誤: ALL_ENTITY_TYPES_STR 未定義，無法進行 tokenization。請先執行 Cell 2。")
    else:
        tokenized_dataset_model1 = dataset_model1.map(
            tokenize_function_causal_lm, # 使用新的 tokenization 函數
            batched=True, 
            remove_columns=["id", "input_text", "target_text"]
        )
        print("\n模型1資料集 Tokenization 完成 (Causal LM 格式)。")
        print(tokenized_dataset_model1)
else:
    tokenized_dataset_model1 = None
    print("\n模型1資料集 Tokenization 跳過。")

if tokenizer and dataset_model2:
    if 'ALL_ENTITY_TYPES_STR' not in globals() or not ALL_ENTITY_TYPES_STR:
        print("錯誤: ALL_ENTITY_TYPES_STR 未定義，無法進行 tokenization。請先執行 Cell 2。")
    else:
        tokenized_dataset_model2 = dataset_model2.map(
            tokenize_function_causal_lm, # 使用新的 tokenization 函數
            batched=True, 
            remove_columns=["id", "input_text", "target_text"]
        )
        print("\n模型2資料集 Tokenization 完成 (Causal LM 格式)。")
        print(tokenized_dataset_model2)
else:
    tokenized_dataset_model2 = None
    print("\n模型2資料集 Tokenization 跳過。")

Tokenizer './models/Qwen3-8B' 載入成功。Pad token ID: 151643


Map: 100%|██████████| 232/232 [00:02<00:00, 103.90 examples/s]



模型1資料集 Tokenization 完成 (Causal LM 格式)。
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2082
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 232
    })
})


Map: 100%|██████████| 272/272 [00:02<00:00, 103.84 examples/s]


模型2資料集 Tokenization 完成 (Causal LM 格式)。
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2445
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 272
    })
})


In [6]:
# Cell 6: 模型載入與 PEFT (LoRA) 設定

def load_model_for_peft(model_path):
    if not tokenizer: # Tokenizer 在 Cell 5 載入
        print("Tokenizer 未載入，無法繼續載入模型。")
        return None

    # 為 PEFT 微調載入基礎模型 (例如使用4-bit量化)
    # RTX 6000 Ada (48GB) 微調 8B 模型，4-bit + LoRA 應該可行
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",          # nf4 通常效果較好
        bnb_4bit_compute_dtype=torch.bfloat16, # Ada GPU 支援 bfloat16
        bnb_4bit_use_double_quant=True,     # 可選，進一步節省記憶體
    )
    
    print(f"\n正在從 '{model_path}' 以4-bit模式載入基礎模型用於PEFT...")
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16, # 與 bnb_4bit_compute_dtype 一致或省略
            device_map="auto", # Accelerate 會處理設備映射
            trust_remote_code=True,
            attn_implementation="sdpa" # 保持以防 flash-attn 問題
        )
        print("基礎模型載入成功。")

        # PEFT 前的準備
        model = prepare_model_for_kbit_training(model)

        # LoRA 設定
        # Qwen 模型的 target_modules 可能需要查閱其設定檔或社群討論確定最優選擇
        # LORA_TARGET_MODULES 已在 Cell 2 中定義
        peft_config = LoraConfig(
            r=LORA_R,
            lora_alpha=LORA_ALPHA,
            lora_dropout=LORA_DROPOUT,
            target_modules=LORA_TARGET_MODULES,
            bias="none",
            task_type=TaskType.CAUSAL_LM # 因為我們是讓 Causal LM 生成目標序列
        )

        peft_model = get_peft_model(model, peft_config)
        print("\nPEFT Model (LoRA) 創建成功。")
        peft_model.print_trainable_parameters()
        return peft_model
        
    except Exception as e:
        print(f"載入基礎模型或創建 PEFT 模型失敗: {e}")
        return None

# 稍後在訓練時，我們會為 Model 1 和 Model 2 分別調用此函數
print("模型載入與PEFT設定函式定義完成。")

模型載入與PEFT設定函式定義完成。


In [7]:
# Cell 7: 訓練函式

def train_ner_model(
    model_name_for_output, 
    peft_model_to_train, 
    tokenized_train_dataset, 
    tokenized_eval_dataset,
    num_epochs,
    batch_size,
    learning_rate,
    output_dir
):
    if not peft_model_to_train or not tokenized_train_dataset:
        print("模型或訓練資料集未準備好，無法開始訓練。")
        return None

    print(f"\n--- 開始訓練模型: {model_name_for_output} ---")
    print(f"  輸出目錄: {output_dir}")
    print(f"  Epochs: {num_epochs}, Batch Size: {batch_size}, LR: {learning_rate}")

    # Data Collator for Seq2Seq (Causal LM 也適用)
    # pad_to_multiple_of=8 有時對性能有益
    data_collator = DataCollatorForSeq2Seq(
        tokenizer, 
        model=peft_model_to_train, 
        label_pad_token_id=tokenizer.pad_token_id, # 確保標籤也被正確 padding
        pad_to_multiple_of=8
    )

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        weight_decay=0.01,
        logging_strategy="epoch",
        eval_strategy="epoch" if tokenized_eval_dataset else "no",
        save_strategy="epoch",
        load_best_model_at_end=True if tokenized_eval_dataset else False,
        report_to="none", # 可設為 "tensorboard", "wandb" 等
        fp16=False, # 如果基礎模型是 bfloat16，這裡通常設 False
        bf16=True if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else False, # 在 Ada GPU 上啟用 bf16
        gradient_accumulation_steps=4, # 模擬更大批次大小，可調整
        lr_scheduler_type="cosine",
        warmup_ratio=0.05,
    )

    trainer = Trainer(
        model=peft_model_to_train,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    print("\n開始訓練...")
    try:
        trainer.train()
        print("訓練完成。")
        
        # 保存最終的 LoRA 適配器
        final_adapter_path = os.path.join(output_dir, "final_adapter")
        peft_model_to_train.save_pretrained(final_adapter_path)
        tokenizer.save_pretrained(final_adapter_path) # 保存 tokenizer 配置
        print(f"最終 LoRA 適配器已保存到: {final_adapter_path}")
        return final_adapter_path
    except Exception as e:
        print(f"訓練過程中發生錯誤: {e}")
        return None

print("訓練函式定義完成。")

訓練函式定義完成。


In [13]:
# Cell 8: 訓練模型 1 (僅原始資料)

if tokenized_dataset_model1 and tokenizer: # 確保 tokenizer 也已成功載入
    print("\n--- 準備訓練模型 1 (僅原始資料) ---")
    peft_model1 = load_model_for_peft(BASE_MODEL_PATH) # 每次訓練都重新載入基礎模型和 LoRA 配置
    
    if peft_model1:
        trained_adapter_path_model1 = train_ner_model(
            model_name_for_output="Qwen3-8B_NER_OriginalData",
            peft_model_to_train=peft_model1,
            tokenized_train_dataset=tokenized_dataset_model1["train"],
            tokenized_eval_dataset=tokenized_dataset_model1.get("validation"), # Handle if no validation set
            num_epochs=NUM_TRAIN_EPOCHS_MODEL1,
            batch_size=PER_DEVICE_TRAIN_BATCH_SIZE_MODEL1,
            learning_rate=LEARNING_RATE_MODEL1,
            output_dir=OUTPUT_DIR_MODEL1
        )
        if trained_adapter_path_model1:
            print(f"模型1 LoRA 適配器訓練完成並保存於: {trained_adapter_path_model1}")
        else:
            print("模型1訓練失敗。")
    else:
        print("模型1的基礎模型或PEFT模型未能成功載入，跳過訓練。")
else:
    print("模型1的資料集或 Tokenizer 未準備好，跳過訓練。")

# 清理CUDA緩存（可選，但有助於接下來訓練模型2）
if torch.cuda.is_available():
    del peft_model1 # 嘗試釋放模型1的資源
    torch.cuda.empty_cache()
    print("\n釋放模型1資源並清理CUDA快取。")


--- 準備訓練模型 1 (僅原始資料) ---

正在從 './models/Qwen3-8B' 以4-bit模式載入基礎模型用於PEFT...


Loading checkpoint shards: 100%|██████████| 5/5 [01:18<00:00, 15.65s/it]


基礎模型載入成功。


C:\Users\ISLAB-121\AppData\Local\Temp\ipykernel_14292\1732842693.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



PEFT Model (LoRA) 創建成功。
trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301

--- 開始訓練模型: Qwen3-8B_NER_OriginalData ---
  輸出目錄: ./qwen3_8b_ner_original_data_non_fp8
  Epochs: 3, Batch Size: 2, LR: 0.0002

開始訓練...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\ISLAB-121\.conda\envs\aicup\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.652700,0.285419
2,0.194300,0.237638
3,0.096600,0.249389


c:\Users\ISLAB-121\.conda\envs\aicup\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\ISLAB-121\.conda\envs\aicup\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


訓練完成。
最終 LoRA 適配器已保存到: ./qwen3_8b_ner_original_data_non_fp8\final_adapter
模型1 LoRA 適配器訓練完成並保存於: ./qwen3_8b_ner_original_data_non_fp8\final_adapter

釋放模型1資源並清理CUDA快取。


In [8]:
# Cell 9: 訓練模型 2 (原始資料 + 增強資料)

if tokenized_dataset_model2 and tokenizer:
    print("\n--- 準備訓練模型 2 (原始資料 + 增強資料) ---")
    # 為模型2重新載入基礎模型和 LoRA 配置，以確保是乾淨的狀態
    peft_model2 = load_model_for_peft(BASE_MODEL_PATH) 
    
    if peft_model2:
        trained_adapter_path_model2 = train_ner_model(
            model_name_for_output="Qwen3-8B_NER_AugmentedData",
            peft_model_to_train=peft_model2,
            tokenized_train_dataset=tokenized_dataset_model2["train"],
            tokenized_eval_dataset=tokenized_dataset_model2.get("validation"),
            num_epochs=NUM_TRAIN_EPOCHS_MODEL2,
            batch_size=PER_DEVICE_TRAIN_BATCH_SIZE_MODEL2,
            learning_rate=LEARNING_RATE_MODEL2,
            output_dir=OUTPUT_DIR_MODEL2
        )
        if trained_adapter_path_model2:
            print(f"模型2 LoRA 適配器訓練完成並保存於: {trained_adapter_path_model2}")
        else:
            print("模型2訓練失敗。")
    else:
        print("模型2的基礎模型或PEFT模型未能成功載入，跳過訓練。")
else:
    print("模型2的資料集或 Tokenizer 未準備好，跳過訓練。")

if torch.cuda.is_available():
    del peft_model2
    torch.cuda.empty_cache()
    print("\n釋放模型2資源並清理CUDA快取。")


--- 準備訓練模型 2 (原始資料 + 增強資料) ---

正在從 './models/Qwen3-8B' 以4-bit模式載入基礎模型用於PEFT...


Loading checkpoint shards: 100%|██████████| 5/5 [01:46<00:00, 21.26s/it]


基礎模型載入成功。


C:\Users\ISLAB-121\AppData\Local\Temp\ipykernel_12336\1732842693.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



PEFT Model (LoRA) 創建成功。
trainable params: 43,646,976 || all params: 8,234,382,336 || trainable%: 0.5301

--- 開始訓練模型: Qwen3-8B_NER_AugmentedData ---
  輸出目錄: ./qwen3_8b_ner_augmented_data_non_fp8
  Epochs: 3, Batch Size: 2, LR: 0.0002

開始訓練...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\ISLAB-121\.conda\envs\aicup\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.665400,0.301463
2,0.184900,0.259415
3,0.091800,0.271553


c:\Users\ISLAB-121\.conda\envs\aicup\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\ISLAB-121\.conda\envs\aicup\lib\site-packages\torch\_dynamo\eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


訓練完成。
最終 LoRA 適配器已保存到: ./qwen3_8b_ner_augmented_data_non_fp8\final_adapter
模型2 LoRA 適配器訓練完成並保存於: ./qwen3_8b_ner_augmented_data_non_fp8\final_adapter

釋放模型2資源並清理CUDA快取。


In [12]:
# Cell 10: (可獨立運行版本) NER 推斷、時間戳對齊與格式化輸出 (包含增強版對齊邏輯)

import os
import re
import torch
import math 
import Levenshtein # 用於模糊匹配
from collections import Counter, defaultdict # 如果需要重新實現 get_all_entity_types (此版本不需要)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

print("--- Cell 10 (獨立運行版本): NER 推斷、時間戳對齊與格式化輸出 ---")

# --- 1. 重要參數設定 (請務必根據您的環境和訓練結果修改這些路徑和變數) ---

# 基礎 Qwen 模型路徑 (與您訓練時使用的基礎模型一致)
BASE_MODEL_PATH = "./models/Qwen3-8B"  # <--- 請確認或修改為您的 Qwen3-8B 模型路徑

# 已訓練好的 NER 模型 LoRA 適配器路徑
# 這應該指向您在之前 Cell (例如 Cell 8 或 Cell 9) 訓練後保存的 "final_adapter" 文件夾
# 範例：假設您要使用第一個模型 (僅原始資料訓練) 的結果
OUTPUT_DIR_MODEL1_FROM_TRAINING = "./qwen3_8b_ner_original_data_non_fp8" # <--- 確認這是您模型1的輸出目錄
NER_MODEL_ADAPTER_PATH = os.path.join(OUTPUT_DIR_MODEL1_FROM_TRAINING, "final_adapter") 
# 或者，如果您想使用模型2 (原始+增強資料訓練) 的結果:
# OUTPUT_DIR_MODEL2_FROM_TRAINING = "./qwen3_8b_ner_augmented_data_non_fp8" # <--- 確認這是您模型2的輸出目錄
# NER_MODEL_ADAPTER_PATH = os.path.join(OUTPUT_DIR_MODEL2_FROM_TRAINING, "final_adapter")

# 輸入的帶有詞級時間戳的轉錄檔案
INPUT_FILE_WITH_TIMESTAMPS = "transcription_with_timestamps_original_model.txt" # <--- 確認此輸入檔案存在且路徑正確

# 最終輸出檔案的路徑
FINAL_PREDICTIONS_OUTPUT_FILE = "ner_predictions_aligned_final.txt" # 您可以自訂輸出檔名

# NER 任務相關的特殊字串 (應與訓練時使用的設定一致)
NULL_ENTITY_REPRESENTATION = "Null"
ENTITY_SEPARATOR = "##"
TYPE_TEXT_SEPARATOR = "\t"

# **** 非常重要：所有可能的實體類型列表 (字串格式，用逗號加空格分隔) ****
# 這個列表應該包含您模型訓練時看到的所有實體類型。
# 您可以從之前 Cell 2 的 ALL_ENTITY_TYPES_STR 變數複製過來，或者手動填寫。
# 這裡我先使用您之前分析得到的類型列表作為範例，請務必根據您訓練時的實際情況核對或修改！
ALL_ENTITY_TYPES_LIST = [
    "PATIENT", "DOCTOR", "USERNAME", "FAMILYNAME", "PERSONALNAME",
    "PROFESSION",
    "ROOM", "DEPARTMENT", "HOSPITAL", "ORGANIZATION", "STREET", "CITY", 
    "DISTRICT", "COUNTY", "STATE", "COUNTRY", "ZIP", "LOCATION-OTHER",
    "AGE",
    "DATE", "TIME", "DURATION", "SET",
    "PHONE", "FAX", "EMAIL", "URL", "IPADDRESS",
    "SOCIAL_SECURITY_NUMBER", "MEDICAL_RECORD_NUMBER", "HEALTH_PLAN_NUMBER", 
    "ACCOUNT_NUMBER", "LICENSE_NUMBER", "VEHICLE_ID", "DEVICE_ID", 
    "BIOMETRIC_ID", "ID_NUMBER",
    "OTHER"
]
ALL_ENTITY_TYPES_STR = ", ".join(sorted(list(set(ALL_ENTITY_TYPES_LIST)))) # 確保唯一並排序

# 運算設備設定
DEVICE_FOR_MODEL = "cuda" if torch.cuda.is_available() else "cpu"

print(f"基礎模型路徑: {BASE_MODEL_PATH}")
print(f"NER 模型適配器路徑: {NER_MODEL_ADAPTER_PATH}")
print(f"輸入的帶時間戳文本檔案: {INPUT_FILE_WITH_TIMESTAMPS}")
print(f"最終輸出檔案: {FINAL_PREDICTIONS_OUTPUT_FILE}")
print(f"將使用的運算設備: {DEVICE_FOR_MODEL}")
print(f"預期提取的實體類型: {ALL_ENTITY_TYPES_STR}")
print(f"無實體時的表示: {NULL_ENTITY_REPRESENTATION}")


# --- 2. Tokenizer 載入 ---
tokenizer = None
print(f"\n[Tokenizer載入] 正在從 '{BASE_MODEL_PATH}' 載入 Tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"  Tokenizer pad_token 未設定，已設為 eos_token ('{tokenizer.eos_token}')")
    print(f"  Tokenizer 載入成功。Pad token ID: {tokenizer.pad_token_id}")
except Exception as e:
    print(f"  載入 Tokenizer '{BASE_MODEL_PATH}' 失敗: {e}")
    tokenizer = None 

# --- 3. 輔助函數定義 ---

def parse_input_line_with_timestamps(line_from_file):
    """解析 'filename\tWord[start-end]Word[start-end]...' 格式的行。"""
    parts = line_from_file.strip().split('\t', 1)
    if len(parts) != 2:
        return None, None, None
    filename = parts[0]
    timed_content = parts[1]
    word_time_objects = []
    clean_sentence_parts = []
    matches = re.findall(r"(.+?)\[([\d\.]+)-([\d\.]+)\]", timed_content)
    if not matches and timed_content:
        cleaned_text = re.sub(r"\[[\d\.]+-[\d\.]+\]", "", timed_content).strip()
        return filename, cleaned_text, [] 
    for match_tuple in matches:
        word, start_str, end_str = match_tuple
        try:
            word_cleaned = word.replace("[", "").replace("]", "").strip()
            if not word_cleaned: continue
            word_time_objects.append({"text": word_cleaned, "start": float(start_str), "end": float(end_str)})
            clean_sentence_parts.append(word_cleaned)
        except ValueError:
            clean_sentence_parts.append(word_cleaned)
    return filename, " ".join(clean_sentence_parts), word_time_objects

def align_entity_flexible_gaps(
    predicted_entity_text,
    word_time_objects_list,
    search_start_sentence_idx=0,
    max_edit_distance_ratio=0.30, 
    absolute_max_edit_distance_for_short_words=1, 
    max_intervening_words=1 
):
    """將 NER 預測實體對齊到帶時間戳的詞語列表 (允許間隔和模糊匹配)。"""
    if not predicted_entity_text or not word_time_objects_list:
        return None, None, search_start_sentence_idx

    norm_pred_text = re.sub(r'[^\w\s-]', '', predicted_entity_text.lower()).strip()
    pred_entity_words = [w for w in norm_pred_text.split(' ') if w]

    if not pred_entity_words:
        return None, None, search_start_sentence_idx

    num_pred_words = len(pred_entity_words)
    num_sentence_words = len(word_time_objects_list)

    for i in range(search_start_sentence_idx, num_sentence_words):
        sentence_word_i_obj = word_time_objects_list[i]
        norm_sent_word_i = re.sub(r'[^\w\s-]', '', sentence_word_i_obj["text"].lower()).strip()
        
        distance_first_word = Levenshtein.distance(pred_entity_words[0], norm_sent_word_i)
        len_pred_w0 = len(pred_entity_words[0])
        max_dist_w0 = absolute_max_edit_distance_for_short_words if len_pred_w0 <= 4 else math.ceil(len_pred_w0 * max_edit_distance_ratio)

        if distance_first_word <= max_dist_w0:
            matched_sentence_word_indices = [i]
            if num_pred_words == 1:
                start_time = word_time_objects_list[i]["start"]
                end_time = word_time_objects_list[i]["end"]
                next_search_start_idx_overall = i + 1
                return start_time, end_time, next_search_start_idx_overall

            current_last_matched_sentence_idx = i 
            all_pred_words_found = True
            for k_pred_idx in range(1, num_pred_words):
                pred_word_k = pred_entity_words[k_pred_idx]
                found_kth_pred_word = False
                search_window_start = current_last_matched_sentence_idx + 1
                search_window_end = min(current_last_matched_sentence_idx + 1 + max_intervening_words + 1, num_sentence_words)
                for search_idx_in_window in range(search_window_start, search_window_end):
                    sentence_word_obj_in_window = word_time_objects_list[search_idx_in_window]
                    norm_sent_word_in_window = re.sub(r'[^\w\s-]', '', sentence_word_obj_in_window["text"].lower()).strip()
                    distance_kth_word = Levenshtein.distance(pred_word_k, norm_sent_word_in_window)
                    len_pred_wk = len(pred_word_k)
                    max_dist_wk = absolute_max_edit_distance_for_short_words if len_pred_wk <= 4 else math.ceil(len_pred_wk * max_edit_distance_ratio)
                    if distance_kth_word <= max_dist_wk:
                        matched_sentence_word_indices.append(search_idx_in_window)
                        current_last_matched_sentence_idx = search_idx_in_window
                        found_kth_pred_word = True; break
                if not found_kth_pred_word: all_pred_words_found = False; break
            
            if all_pred_words_found:
                first_matched_sent_idx = matched_sentence_word_indices[0]
                last_matched_sent_idx = matched_sentence_word_indices[-1]
                start_time = word_time_objects_list[first_matched_sent_idx]["start"]
                end_time = word_time_objects_list[last_matched_sent_idx]["end"]
                next_search_start_idx_overall = last_matched_sent_idx + 1
                return start_time, end_time, next_search_start_idx_overall
    return None, None, search_start_sentence_idx


# --- 4. 主推斷與對齊流程函數 ---
def run_ner_inference_and_write_output(
    input_file_path, 
    model_adapter_path_to_load, 
    base_model_path_for_inference, 
    tokenizer_instance_to_use, 
    output_file_path
):
    if not all([os.path.exists(input_file_path), os.path.exists(model_adapter_path_to_load), 
                os.path.exists(base_model_path_for_inference), tokenizer_instance_to_use]):
        print("錯誤: 一個或多個必要的路徑或物件未提供或無效。")
        if not os.path.exists(input_file_path): print(f"  - 輸入檔案未找到: {input_file_path}")
        if not os.path.exists(model_adapter_path_to_load): print(f"  - 模型適配器未找到: {model_adapter_path_to_load}")
        if not os.path.exists(base_model_path_for_inference): print(f"  - 基礎模型路徑未找到: {base_model_path_for_inference}")
        if not tokenizer_instance_to_use: print("  - Tokenizer 未提供。")
        return

    print(f"\n[推斷流程] 正在載入基礎模型 '{base_model_path_for_inference}'...")
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_use_double_quant=True,
    ) if torch.cuda.is_available() and DEVICE_FOR_MODEL == "cuda" else None
    model_dtype_inference = torch.bfloat16 if torch.cuda.is_available() and DEVICE_FOR_MODEL == "cuda" else torch.float32
    
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path_for_inference, quantization_config=bnb_config_inference,
        torch_dtype=model_dtype_inference, device_map="auto", trust_remote_code=True,
        attn_implementation="sdpa" # 保持以防 flash-attn 問題
    )
    print(f"[推斷流程] 正在從 '{model_adapter_path_to_load}' 載入 LoRA 適配器...")
    ner_model = PeftModel.from_pretrained(base_model, model_adapter_path_to_load)
    ner_model.eval() 
    print("[推斷流程] NER 模型載入完畢並設為評估模式。")

    system_prompt = (
        f"You are an expert Named Entity Recognition (NER) system. "
        f"Your task is to extract all specified entities from the given medical text. "
        f"Output the entities in the format: Type1{TYPE_TEXT_SEPARATOR}Text1{ENTITY_SEPARATOR}Type2{TYPE_TEXT_SEPARATOR}Text2... "
        f"If no entities are found, output exactly '{NULL_ENTITY_REPRESENTATION}'.\n"
        f"The entity types to look for are: {ALL_ENTITY_TYPES_STR}."
    )
    
    results_to_write = []
    processed_line_count = 0
    try:
        with open(input_file_path, 'r', encoding='utf-8') as infile:
            print(f"\n[推斷流程] 開始處理輸入檔案: {input_file_path}")
            for line_idx, line_content in enumerate(infile):
                processed_line_count +=1
                filename, clean_sentence, word_times = parse_input_line_with_timestamps(line_content)
                if not filename or not clean_sentence: print(f"  跳過第 {line_idx+1} 行。"); continue
                if not word_times: print(f"  警告: 檔案 '{filename}' 無詞級時間戳，結果將不含時間。")

                print(f"\n  處理檔案: {filename} (行 {line_idx+1})")
                
                messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": clean_sentence}]
                prompt_string_for_model = tokenizer_instance_to_use.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
                inputs = tokenizer_instance_to_use(prompt_string_for_model, return_tensors="pt", padding=False).to(ner_model.device)

                with torch.no_grad():
                    outputs = ner_model.generate(
                        **inputs, max_new_tokens=256, temperature=0.1, do_sample=False,
                        eos_token_id=tokenizer_instance_to_use.eos_token_id,
                        pad_token_id=tokenizer_instance_to_use.pad_token_id
                    )
                input_length = inputs.input_ids.shape[1]; generated_tokens = outputs[0][input_length:]
                predicted_ner_string = tokenizer_instance_to_use.decode(generated_tokens, skip_special_tokens=True).strip()
                # predicted_ner_string = predicted_ner_string.replace("<|im_end|>", "").strip() # Qwen 特定清理
                print(f"    模型原始輸出: {predicted_ner_string}")

                if predicted_ner_string.lower() == NULL_ENTITY_REPRESENTATION.lower() or not predicted_ner_string:
                    print(f"    未預測到實體。"); continue 
                
                predicted_entity_parts = predicted_ner_string.split(ENTITY_SEPARATOR)
                current_word_search_offset_in_sentence = 0 
                
                for entity_part_str in predicted_entity_parts:
                    type_text_pair_parts = entity_part_str.split(TYPE_TEXT_SEPARATOR, 1)
                    if len(type_text_pair_parts) == 2:
                        entity_type, entity_text = type_text_pair_parts[0].strip(), type_text_pair_parts[1].strip()
                        if entity_type and entity_text:
                            start_ts, end_ts = None, None
                            if word_times: 
                                start_ts, end_ts, next_search_offset = align_entity_flexible_gaps(
                                    entity_text, word_times, current_word_search_offset_in_sentence
                                )
                                if start_ts is not None: 
                                    current_word_search_offset_in_sentence = next_search_offset 
                                else:
                                    print(f"      [對齊失敗] 實體 '{entity_text}' (Type: {entity_type}) 從詞索引 {current_word_search_offset_in_sentence} 開始未找到匹配。")
                            
                            start_ts_str = f"{start_ts:.3f}" if start_ts is not None else "None"
                            end_ts_str = f"{end_ts:.3f}" if end_ts is not None else "None"
                            output_line = f"{filename}\t{entity_type}\t{start_ts_str}\t{end_ts_str}\t{entity_text}\n"
                            results_to_write.append(output_line)
                            print(f"      輸出: {output_line.strip()}")
                        else: print(f"    警告: 解析實體部分 '{entity_part_str}' 失敗 (Type或Text為空)。")
                    else: print(f"    警告: 解析實體部分 '{entity_part_str}' 失敗 (無法按預期分隔符分割)。")
            
            with open(output_file_path, 'w', encoding='utf-8') as outfile:
                for out_l in results_to_write: outfile.write(out_l)
            print(f"\n[推斷流程] 共處理 {processed_line_count} 行。結果已寫入到: {output_file_path}")
            if results_to_write:
                print("\n輸出範例 (前5條):")
                for res_line in results_to_write[:5]: print(res_line.strip())

    except Exception as e:
        print(f"[推斷流程] 執行過程中發生嚴重錯誤: {e}")
        import traceback
        traceback.print_exc()


# --- 5. 執行推斷 ---
if tokenizer is not None and \
   os.path.exists(BASE_MODEL_PATH) and \
   ALL_ENTITY_TYPES_STR and \
   DEVICE_FOR_MODEL: # 確保所有依賴都已定義
    
    if os.path.exists(NER_MODEL_ADAPTER_PATH):
        print(f"\n開始使用適配器 '{NER_MODEL_ADAPTER_PATH}' 進行推斷...")
        run_ner_inference_and_write_output(
            INPUT_FILE_WITH_TIMESTAMPS,
            NER_MODEL_ADAPTER_PATH,
            BASE_MODEL_PATH, 
            tokenizer,       
            FINAL_PREDICTIONS_OUTPUT_FILE
        )
    else:
        print(f"\n錯誤: 指定的 NER 模型適配器路徑 '{NER_MODEL_ADAPTER_PATH}' 不存在。無法執行推斷。")
else:
    print("\n錯誤: 必須的組件（Tokenizer, 基礎模型路徑, 實體類型列表, 或設備資訊）未正確定義或準備好。無法執行推斷。")
    if tokenizer is None: print("  - 原因: Tokenizer 未成功載入。")
    if not os.path.exists(BASE_MODEL_PATH): print(f"  - 原因: 基礎模型路徑 '{BASE_MODEL_PATH}' 不存在。")
    if not ALL_ENTITY_TYPES_STR: print("  - 原因: ALL_ENTITY_TYPES_STR 為空或未定義 (請在 Cell 開頭設定)。")
    if 'DEVICE_FOR_MODEL' not in globals() or not DEVICE_FOR_MODEL : print(" - 原因: DEVICE_FOR_MODEL 未定義。")


print("\n--- Cell 10 (獨立運行版本): 推斷流程結束 ---")

--- Cell 10 (獨立運行版本): NER 推斷、時間戳對齊與格式化輸出 ---
基礎模型路徑: ./models/Qwen3-8B
NER 模型適配器路徑: ./qwen3_8b_ner_original_data_non_fp8\final_adapter
輸入的帶時間戳文本檔案: transcription_with_timestamps_original_model.txt
最終輸出檔案: ner_predictions_aligned_final.txt
將使用的運算設備: cuda
預期提取的實體類型: ACCOUNT_NUMBER, AGE, BIOMETRIC_ID, CITY, COUNTRY, COUNTY, DATE, DEPARTMENT, DEVICE_ID, DISTRICT, DOCTOR, DURATION, EMAIL, FAMILYNAME, FAX, HEALTH_PLAN_NUMBER, HOSPITAL, ID_NUMBER, IPADDRESS, LICENSE_NUMBER, LOCATION-OTHER, MEDICAL_RECORD_NUMBER, ORGANIZATION, OTHER, PATIENT, PERSONALNAME, PHONE, PROFESSION, ROOM, SET, SOCIAL_SECURITY_NUMBER, STATE, STREET, TIME, URL, USERNAME, VEHICLE_ID, ZIP
無實體時的表示: Null

[Tokenizer載入] 正在從 './models/Qwen3-8B' 載入 Tokenizer...
  Tokenizer 載入成功。Pad token ID: 151643

開始使用適配器 './qwen3_8b_ner_original_data_non_fp8\final_adapter' 進行推斷...

[推斷流程] 正在載入基礎模型 './models/Qwen3-8B'...


Loading checkpoint shards: 100%|██████████| 5/5 [01:41<00:00, 20.28s/it]


[推斷流程] 正在從 './qwen3_8b_ner_original_data_non_fp8\final_adapter' 載入 LoRA 適配器...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[推斷流程] NER 模型載入完畢並設為評估模式。

[推斷流程] 開始處理輸入檔案: transcription_with_timestamps_original_model.txt

  處理檔案: 60014 (行 1)


...

    模型原始輸出: PATIENT	于桂芝##DATE	民国一零七年九月五日##TIME	下午##DATE	今天##HOSPITAL	东方医院##DOCTOR	南祥辉
      [對齊失敗] 實體 '于桂芝' (Type: PATIENT) 從詞索引 0 開始未找到匹配。
      輸出: 84778	PATIENT	None	None	于桂芝
      [對齊失敗] 實體 '民国一零七年九月五日' (Type: DATE) 從詞索引 0 開始未找到匹配。
      輸出: 84778	DATE	None	None	民国一零七年九月五日
      輸出: 84778	TIME	14.150	14.170	下午
      輸出: 84778	DATE	16.280	16.550	今天
      [對齊失敗] 實體 '东方医院' (Type: HOSPITAL) 從詞索引 71 開始未找到匹配。
      輸出: 84778	HOSPITAL	None	None	东方医院
      [對齊失敗] 實體 '南祥辉' (Type: DOCTOR) 從詞索引 71 開始未找到匹配。
      輸出: 84778	DOCTOR	None	None	南祥辉

  處理檔案: 84870 (行 708)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    模型原始輸出: PATIENT	赵紫衣##PATIENT	紫妮
      [對齊失敗] 實體 '赵紫衣' (Type: PATIENT) 從詞索引 0 開始未找到匹配。
      輸出: 84870	PATIENT	None	None	赵紫衣
      輸出: 84870	PATIENT	0.450	0.610	紫妮

  處理檔案: 84871 (行 709)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    模型原始輸出: PATIENT	云 昊
      輸出: 84871	PATIENT	0.000	1.400	云 昊

  處理檔案: 84875 (行 710)
    模型原始輸出: DURATION	很久##FAMILYNAME	天宇##FAMILYNAME	天羽
      輸出: 84875	DURATION	6.760	7.050	很久
      輸出: 84875	FAMILYNAME	10.000	10.650	天宇
      輸出: 84875	FAMILYNAME	12.450	12.710	天羽

[推斷流程] 共處理 710 行。結果已寫入到: ner_predictions_aligned_final.txt

輸出範例 (前5條):
60014	DATE	3.030	4.550	July 19, 2062
60014	DATE	5.880	6.560	July 28, 262
60014	PATIENT	12.980	13.380	Amanda
60015	DOCTOR	2.050	3.050	T. Wernz
60015	DATE	6.410	9.340	June 6, 2063

--- Cell 10 (獨立運行版本): 推斷流程結束 ---


In [13]:
# Cell 10: (可獨立運行版本) NER 推斷、時間戳對齊與格式化輸出 (包含增強版對齊邏輯)

import os
import re
import torch
import math 
import Levenshtein # 用於模糊匹配
from collections import Counter, defaultdict # 如果需要重新實現 get_all_entity_types (此版本不需要)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

print("--- Cell 10 (獨立運行版本): NER 推斷、時間戳對齊與格式化輸出 ---")

# --- 1. 重要參數設定 (請務必根據您的環境和訓練結果修改這些路徑和變數) ---

# 基礎 Qwen 模型路徑 (與您訓練時使用的基礎模型一致)
BASE_MODEL_PATH = "./models/Qwen3-8B"  # <--- 請確認或修改為您的 Qwen3-8B 模型路徑

# 已訓練好的 NER 模型 LoRA 適配器路徑
# 這應該指向您在之前 Cell (例如 Cell 8 或 Cell 9) 訓練後保存的 "final_adapter" 文件夾
# 範例：假設您要使用第一個模型 (僅原始資料訓練) 的結果
# OUTPUT_DIR_MODEL1_FROM_TRAINING = "./qwen3_8b_ner_original_data_non_fp8" # <--- 確認這是您模型1的輸出目錄
# NER_MODEL_ADAPTER_PATH = os.path.join(OUTPUT_DIR_MODEL1_FROM_TRAINING, "final_adapter") 
# 或者，如果您想使用模型2 (原始+增強資料訓練) 的結果:
OUTPUT_DIR_MODEL2_FROM_TRAINING = "./qwen3_8b_ner_augmented_data_non_fp8" # <--- 確認這是您模型2的輸出目錄
NER_MODEL_ADAPTER_PATH = os.path.join(OUTPUT_DIR_MODEL2_FROM_TRAINING, "final_adapter")

# 輸入的帶有詞級時間戳的轉錄檔案
INPUT_FILE_WITH_TIMESTAMPS = "transcription_with_timestamps_original_model.txt" # <--- 確認此輸入檔案存在且路徑正確

# 最終輸出檔案的路徑
FINAL_PREDICTIONS_OUTPUT_FILE = "augmented_ner_predictions_aligned_final.txt" # 您可以自訂輸出檔名

# NER 任務相關的特殊字串 (應與訓練時使用的設定一致)
NULL_ENTITY_REPRESENTATION = "Null"
ENTITY_SEPARATOR = "##"
TYPE_TEXT_SEPARATOR = "\t"

# **** 非常重要：所有可能的實體類型列表 (字串格式，用逗號加空格分隔) ****
# 這個列表應該包含您模型訓練時看到的所有實體類型。
# 您可以從之前 Cell 2 的 ALL_ENTITY_TYPES_STR 變數複製過來，或者手動填寫。
# 這裡我先使用您之前分析得到的類型列表作為範例，請務必根據您訓練時的實際情況核對或修改！
ALL_ENTITY_TYPES_LIST = [
    "PATIENT", "DOCTOR", "USERNAME", "FAMILYNAME", "PERSONALNAME",
    "PROFESSION",
    "ROOM", "DEPARTMENT", "HOSPITAL", "ORGANIZATION", "STREET", "CITY", 
    "DISTRICT", "COUNTY", "STATE", "COUNTRY", "ZIP", "LOCATION-OTHER",
    "AGE",
    "DATE", "TIME", "DURATION", "SET",
    "PHONE", "FAX", "EMAIL", "URL", "IPADDRESS",
    "SOCIAL_SECURITY_NUMBER", "MEDICAL_RECORD_NUMBER", "HEALTH_PLAN_NUMBER", 
    "ACCOUNT_NUMBER", "LICENSE_NUMBER", "VEHICLE_ID", "DEVICE_ID", 
    "BIOMETRIC_ID", "ID_NUMBER",
    "OTHER"
]
ALL_ENTITY_TYPES_STR = ", ".join(sorted(list(set(ALL_ENTITY_TYPES_LIST)))) # 確保唯一並排序

# 運算設備設定
DEVICE_FOR_MODEL = "cuda" if torch.cuda.is_available() else "cpu"

print(f"基礎模型路徑: {BASE_MODEL_PATH}")
print(f"NER 模型適配器路徑: {NER_MODEL_ADAPTER_PATH}")
print(f"輸入的帶時間戳文本檔案: {INPUT_FILE_WITH_TIMESTAMPS}")
print(f"最終輸出檔案: {FINAL_PREDICTIONS_OUTPUT_FILE}")
print(f"將使用的運算設備: {DEVICE_FOR_MODEL}")
print(f"預期提取的實體類型: {ALL_ENTITY_TYPES_STR}")
print(f"無實體時的表示: {NULL_ENTITY_REPRESENTATION}")


# --- 2. Tokenizer 載入 ---
tokenizer = None
print(f"\n[Tokenizer載入] 正在從 '{BASE_MODEL_PATH}' 載入 Tokenizer...")
try:
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"  Tokenizer pad_token 未設定，已設為 eos_token ('{tokenizer.eos_token}')")
    print(f"  Tokenizer 載入成功。Pad token ID: {tokenizer.pad_token_id}")
except Exception as e:
    print(f"  載入 Tokenizer '{BASE_MODEL_PATH}' 失敗: {e}")
    tokenizer = None 

# --- 3. 輔助函數定義 ---

def parse_input_line_with_timestamps(line_from_file):
    """解析 'filename\tWord[start-end]Word[start-end]...' 格式的行。"""
    parts = line_from_file.strip().split('\t', 1)
    if len(parts) != 2:
        return None, None, None
    filename = parts[0]
    timed_content = parts[1]
    word_time_objects = []
    clean_sentence_parts = []
    matches = re.findall(r"(.+?)\[([\d\.]+)-([\d\.]+)\]", timed_content)
    if not matches and timed_content:
        cleaned_text = re.sub(r"\[[\d\.]+-[\d\.]+\]", "", timed_content).strip()
        return filename, cleaned_text, [] 
    for match_tuple in matches:
        word, start_str, end_str = match_tuple
        try:
            word_cleaned = word.replace("[", "").replace("]", "").strip()
            if not word_cleaned: continue
            word_time_objects.append({"text": word_cleaned, "start": float(start_str), "end": float(end_str)})
            clean_sentence_parts.append(word_cleaned)
        except ValueError:
            clean_sentence_parts.append(word_cleaned)
    return filename, " ".join(clean_sentence_parts), word_time_objects

def align_entity_flexible_gaps(
    predicted_entity_text,
    word_time_objects_list,
    search_start_sentence_idx=0,
    max_edit_distance_ratio=0.30, 
    absolute_max_edit_distance_for_short_words=1, 
    max_intervening_words=1 
):
    """將 NER 預測實體對齊到帶時間戳的詞語列表 (允許間隔和模糊匹配)。"""
    if not predicted_entity_text or not word_time_objects_list:
        return None, None, search_start_sentence_idx

    norm_pred_text = re.sub(r'[^\w\s-]', '', predicted_entity_text.lower()).strip()
    pred_entity_words = [w for w in norm_pred_text.split(' ') if w]

    if not pred_entity_words:
        return None, None, search_start_sentence_idx

    num_pred_words = len(pred_entity_words)
    num_sentence_words = len(word_time_objects_list)

    for i in range(search_start_sentence_idx, num_sentence_words):
        sentence_word_i_obj = word_time_objects_list[i]
        norm_sent_word_i = re.sub(r'[^\w\s-]', '', sentence_word_i_obj["text"].lower()).strip()
        
        distance_first_word = Levenshtein.distance(pred_entity_words[0], norm_sent_word_i)
        len_pred_w0 = len(pred_entity_words[0])
        max_dist_w0 = absolute_max_edit_distance_for_short_words if len_pred_w0 <= 4 else math.ceil(len_pred_w0 * max_edit_distance_ratio)

        if distance_first_word <= max_dist_w0:
            matched_sentence_word_indices = [i]
            if num_pred_words == 1:
                start_time = word_time_objects_list[i]["start"]
                end_time = word_time_objects_list[i]["end"]
                next_search_start_idx_overall = i + 1
                return start_time, end_time, next_search_start_idx_overall

            current_last_matched_sentence_idx = i 
            all_pred_words_found = True
            for k_pred_idx in range(1, num_pred_words):
                pred_word_k = pred_entity_words[k_pred_idx]
                found_kth_pred_word = False
                search_window_start = current_last_matched_sentence_idx + 1
                search_window_end = min(current_last_matched_sentence_idx + 1 + max_intervening_words + 1, num_sentence_words)
                for search_idx_in_window in range(search_window_start, search_window_end):
                    sentence_word_obj_in_window = word_time_objects_list[search_idx_in_window]
                    norm_sent_word_in_window = re.sub(r'[^\w\s-]', '', sentence_word_obj_in_window["text"].lower()).strip()
                    distance_kth_word = Levenshtein.distance(pred_word_k, norm_sent_word_in_window)
                    len_pred_wk = len(pred_word_k)
                    max_dist_wk = absolute_max_edit_distance_for_short_words if len_pred_wk <= 4 else math.ceil(len_pred_wk * max_edit_distance_ratio)
                    if distance_kth_word <= max_dist_wk:
                        matched_sentence_word_indices.append(search_idx_in_window)
                        current_last_matched_sentence_idx = search_idx_in_window
                        found_kth_pred_word = True; break
                if not found_kth_pred_word: all_pred_words_found = False; break
            
            if all_pred_words_found:
                first_matched_sent_idx = matched_sentence_word_indices[0]
                last_matched_sent_idx = matched_sentence_word_indices[-1]
                start_time = word_time_objects_list[first_matched_sent_idx]["start"]
                end_time = word_time_objects_list[last_matched_sent_idx]["end"]
                next_search_start_idx_overall = last_matched_sent_idx + 1
                return start_time, end_time, next_search_start_idx_overall
    return None, None, search_start_sentence_idx


# --- 4. 主推斷與對齊流程函數 ---
def run_ner_inference_and_write_output(
    input_file_path, 
    model_adapter_path_to_load, 
    base_model_path_for_inference, 
    tokenizer_instance_to_use, 
    output_file_path
):
    if not all([os.path.exists(input_file_path), os.path.exists(model_adapter_path_to_load), 
                os.path.exists(base_model_path_for_inference), tokenizer_instance_to_use]):
        print("錯誤: 一個或多個必要的路徑或物件未提供或無效。")
        if not os.path.exists(input_file_path): print(f"  - 輸入檔案未找到: {input_file_path}")
        if not os.path.exists(model_adapter_path_to_load): print(f"  - 模型適配器未找到: {model_adapter_path_to_load}")
        if not os.path.exists(base_model_path_for_inference): print(f"  - 基礎模型路徑未找到: {base_model_path_for_inference}")
        if not tokenizer_instance_to_use: print("  - Tokenizer 未提供。")
        return

    print(f"\n[推斷流程] 正在載入基礎模型 '{base_model_path_for_inference}'...")
    bnb_config_inference = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16, bnb_4bit_use_double_quant=True,
    ) if torch.cuda.is_available() and DEVICE_FOR_MODEL == "cuda" else None
    model_dtype_inference = torch.bfloat16 if torch.cuda.is_available() and DEVICE_FOR_MODEL == "cuda" else torch.float32
    
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path_for_inference, quantization_config=bnb_config_inference,
        torch_dtype=model_dtype_inference, device_map="auto", trust_remote_code=True,
        attn_implementation="sdpa" # 保持以防 flash-attn 問題
    )
    print(f"[推斷流程] 正在從 '{model_adapter_path_to_load}' 載入 LoRA 適配器...")
    ner_model = PeftModel.from_pretrained(base_model, model_adapter_path_to_load)
    ner_model.eval() 
    print("[推斷流程] NER 模型載入完畢並設為評估模式。")

    system_prompt = (
        f"You are an expert Named Entity Recognition (NER) system. "
        f"Your task is to extract all specified entities from the given medical text. "
        f"Output the entities in the format: Type1{TYPE_TEXT_SEPARATOR}Text1{ENTITY_SEPARATOR}Type2{TYPE_TEXT_SEPARATOR}Text2... "
        f"If no entities are found, output exactly '{NULL_ENTITY_REPRESENTATION}'.\n"
        f"The entity types to look for are: {ALL_ENTITY_TYPES_STR}."
    )
    
    results_to_write = []
    processed_line_count = 0
    try:
        with open(input_file_path, 'r', encoding='utf-8') as infile:
            print(f"\n[推斷流程] 開始處理輸入檔案: {input_file_path}")
            for line_idx, line_content in enumerate(infile):
                processed_line_count +=1
                filename, clean_sentence, word_times = parse_input_line_with_timestamps(line_content)
                if not filename or not clean_sentence: print(f"  跳過第 {line_idx+1} 行。"); continue
                if not word_times: print(f"  警告: 檔案 '{filename}' 無詞級時間戳，結果將不含時間。")

                print(f"\n  處理檔案: {filename} (行 {line_idx+1})")
                
                messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": clean_sentence}]
                prompt_string_for_model = tokenizer_instance_to_use.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
                inputs = tokenizer_instance_to_use(prompt_string_for_model, return_tensors="pt", padding=False).to(ner_model.device)

                with torch.no_grad():
                    outputs = ner_model.generate(
                        **inputs, max_new_tokens=256, temperature=0.1, do_sample=False,
                        eos_token_id=tokenizer_instance_to_use.eos_token_id,
                        pad_token_id=tokenizer_instance_to_use.pad_token_id
                    )
                input_length = inputs.input_ids.shape[1]; generated_tokens = outputs[0][input_length:]
                predicted_ner_string = tokenizer_instance_to_use.decode(generated_tokens, skip_special_tokens=True).strip()
                # predicted_ner_string = predicted_ner_string.replace("<|im_end|>", "").strip() # Qwen 特定清理
                print(f"    模型原始輸出: {predicted_ner_string}")

                if predicted_ner_string.lower() == NULL_ENTITY_REPRESENTATION.lower() or not predicted_ner_string:
                    print(f"    未預測到實體。"); continue 
                
                predicted_entity_parts = predicted_ner_string.split(ENTITY_SEPARATOR)
                current_word_search_offset_in_sentence = 0 
                
                for entity_part_str in predicted_entity_parts:
                    type_text_pair_parts = entity_part_str.split(TYPE_TEXT_SEPARATOR, 1)
                    if len(type_text_pair_parts) == 2:
                        entity_type, entity_text = type_text_pair_parts[0].strip(), type_text_pair_parts[1].strip()
                        if entity_type and entity_text:
                            start_ts, end_ts = None, None
                            if word_times: 
                                start_ts, end_ts, next_search_offset = align_entity_flexible_gaps(
                                    entity_text, word_times, current_word_search_offset_in_sentence
                                )
                                if start_ts is not None: 
                                    current_word_search_offset_in_sentence = next_search_offset 
                                else:
                                    print(f"      [對齊失敗] 實體 '{entity_text}' (Type: {entity_type}) 從詞索引 {current_word_search_offset_in_sentence} 開始未找到匹配。")
                            
                            start_ts_str = f"{start_ts:.3f}" if start_ts is not None else "None"
                            end_ts_str = f"{end_ts:.3f}" if end_ts is not None else "None"
                            output_line = f"{filename}\t{entity_type}\t{start_ts_str}\t{end_ts_str}\t{entity_text}\n"
                            results_to_write.append(output_line)
                            print(f"      輸出: {output_line.strip()}")
                        else: print(f"    警告: 解析實體部分 '{entity_part_str}' 失敗 (Type或Text為空)。")
                    else: print(f"    警告: 解析實體部分 '{entity_part_str}' 失敗 (無法按預期分隔符分割)。")
            
            with open(output_file_path, 'w', encoding='utf-8') as outfile:
                for out_l in results_to_write: outfile.write(out_l)
            print(f"\n[推斷流程] 共處理 {processed_line_count} 行。結果已寫入到: {output_file_path}")
            if results_to_write:
                print("\n輸出範例 (前5條):")
                for res_line in results_to_write[:5]: print(res_line.strip())

    except Exception as e:
        print(f"[推斷流程] 執行過程中發生嚴重錯誤: {e}")
        import traceback
        traceback.print_exc()


# --- 5. 執行推斷 ---
if tokenizer is not None and \
   os.path.exists(BASE_MODEL_PATH) and \
   ALL_ENTITY_TYPES_STR and \
   DEVICE_FOR_MODEL: # 確保所有依賴都已定義
    
    if os.path.exists(NER_MODEL_ADAPTER_PATH):
        print(f"\n開始使用適配器 '{NER_MODEL_ADAPTER_PATH}' 進行推斷...")
        run_ner_inference_and_write_output(
            INPUT_FILE_WITH_TIMESTAMPS,
            NER_MODEL_ADAPTER_PATH,
            BASE_MODEL_PATH, 
            tokenizer,       
            FINAL_PREDICTIONS_OUTPUT_FILE
        )
    else:
        print(f"\n錯誤: 指定的 NER 模型適配器路徑 '{NER_MODEL_ADAPTER_PATH}' 不存在。無法執行推斷。")
else:
    print("\n錯誤: 必須的組件（Tokenizer, 基礎模型路徑, 實體類型列表, 或設備資訊）未正確定義或準備好。無法執行推斷。")
    if tokenizer is None: print("  - 原因: Tokenizer 未成功載入。")
    if not os.path.exists(BASE_MODEL_PATH): print(f"  - 原因: 基礎模型路徑 '{BASE_MODEL_PATH}' 不存在。")
    if not ALL_ENTITY_TYPES_STR: print("  - 原因: ALL_ENTITY_TYPES_STR 為空或未定義 (請在 Cell 開頭設定)。")
    if 'DEVICE_FOR_MODEL' not in globals() or not DEVICE_FOR_MODEL : print(" - 原因: DEVICE_FOR_MODEL 未定義。")


print("\n--- Cell 10 (獨立運行版本): 推斷流程結束 ---")

--- Cell 10 (獨立運行版本): NER 推斷、時間戳對齊與格式化輸出 ---
基礎模型路徑: ./models/Qwen3-8B
NER 模型適配器路徑: ./qwen3_8b_ner_augmented_data_non_fp8\final_adapter
輸入的帶時間戳文本檔案: transcription_with_timestamps_original_model.txt
最終輸出檔案: augmented_ner_predictions_aligned_final.txt
將使用的運算設備: cuda
預期提取的實體類型: ACCOUNT_NUMBER, AGE, BIOMETRIC_ID, CITY, COUNTRY, COUNTY, DATE, DEPARTMENT, DEVICE_ID, DISTRICT, DOCTOR, DURATION, EMAIL, FAMILYNAME, FAX, HEALTH_PLAN_NUMBER, HOSPITAL, ID_NUMBER, IPADDRESS, LICENSE_NUMBER, LOCATION-OTHER, MEDICAL_RECORD_NUMBER, ORGANIZATION, OTHER, PATIENT, PERSONALNAME, PHONE, PROFESSION, ROOM, SET, SOCIAL_SECURITY_NUMBER, STATE, STREET, TIME, URL, USERNAME, VEHICLE_ID, ZIP
無實體時的表示: Null

[Tokenizer載入] 正在從 './models/Qwen3-8B' 載入 Tokenizer...
  Tokenizer 載入成功。Pad token ID: 151643

開始使用適配器 './qwen3_8b_ner_augmented_data_non_fp8\final_adapter' 進行推斷...

[推斷流程] 正在載入基礎模型 './models/Qwen3-8B'...


Loading checkpoint shards: 100%|██████████| 5/5 [00:13<00:00,  2.77s/it]


[推斷流程] 正在從 './qwen3_8b_ner_augmented_data_non_fp8\final_adapter' 載入 LoRA 適配器...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[推斷流程] NER 模型載入完畢並設為評估模式。

[推斷流程] 開始處理輸入檔案: transcription_with_timestamps_original_model.txt

  處理檔案: 60014 (行 1)


...

    模型原始輸出: PATIENT	桂芝##DATE	民国一零七年九月五日##TIME	下午##DATE	今天##HOSPITAL	东方医院##DOCTOR	南祥辉
      輸出: 84778	PATIENT	0.700	0.940	桂芝
      [對齊失敗] 實體 '民国一零七年九月五日' (Type: DATE) 從詞索引 5 開始未找到匹配。
      輸出: 84778	DATE	None	None	民国一零七年九月五日
      輸出: 84778	TIME	14.150	14.170	下午
      輸出: 84778	DATE	16.280	16.550	今天
      [對齊失敗] 實體 '东方医院' (Type: HOSPITAL) 從詞索引 71 開始未找到匹配。
      輸出: 84778	HOSPITAL	None	None	东方医院
      [對齊失敗] 實體 '南祥辉' (Type: DOCTOR) 從詞索引 71 開始未找到匹配。
      輸出: 84778	DOCTOR	None	None	南祥辉

  處理檔案: 84870 (行 708)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    模型原始輸出: PATIENT	赵紫衣##DOCTOR	马##PATIENT	紫妮
      [對齊失敗] 實體 '赵紫衣' (Type: PATIENT) 從詞索引 0 開始未找到匹配。
      輸出: 84870	PATIENT	None	None	赵紫衣
      輸出: 84870	DOCTOR	0.000	0.430	马
      輸出: 84870	PATIENT	0.450	0.610	紫妮

  處理檔案: 84871 (行 709)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


    模型原始輸出: PATIENT	云 昊
      輸出: 84871	PATIENT	0.000	1.400	云 昊

  處理檔案: 84875 (行 710)
    模型原始輸出: DURATION	很久##FAMILYNAME	天宇##FAMILYNAME	天羽
      輸出: 84875	DURATION	6.760	7.050	很久
      輸出: 84875	FAMILYNAME	10.000	10.650	天宇
      輸出: 84875	FAMILYNAME	12.450	12.710	天羽

[推斷流程] 共處理 710 行。結果已寫入到: augmented_ner_predictions_aligned_final.txt

輸出範例 (前5條):
60014	DATE	3.030	4.550	July 19, 2062
60014	DATE	5.880	6.560	July 28, 262
60014	PATIENT	12.980	13.380	Amanda
60015	DOCTOR	2.050	3.050	T. Wernz
60015	DATE	6.410	9.340	June 6, 2063

--- Cell 10 (獨立運行版本): 推斷流程結束 ---
